In [2]:
import pandas as pd
import arnie
from arnie.utils import *
from arnie.utils import _group_into_non_conflicting_bp

# import csv for pseudoknot predictions

def get_csv(csv_loc):
    df = pd.read_csv(csv_loc)
    return df 

# extract locations for each pseudoknot along with dotbracket structures

def get_info(df):
    
    starts = df['start'].to_list()
    ends = df['end'].to_list()
    sequences = df['sequence'].to_list()
    dotbrackets = df['struct'].to_list()
    
    return starts, ends, sequences, dotbrackets

# import shapeknots data and convert to list

def get_shape_data(filename):
    shape = []
    with open(filename) as f:
        for line in f:
            line = line.strip()
            shape.append(line)
            
    for i in range(len(shape)):
        shape[i] = (-1) if shape[i] == 'nan' else float(shape[i])
        
    return shape

# use Rachael's function to compare shape and dotbracket structure and return ranking

def evaluate_L1_shape_score(s,shape):
    score = 0
    for c,react in zip(s,shape):
        if (c=="." and react>0.25) or (c!="." and react<0.5):
            score += 1
    return score/len(s)

# get locations of pseudoknotted base pairs in a window

def get_groups(dotbracket):
    bp_list = convert_dotbracket_to_bp_list(dotbracket, allow_pseudoknots=True)
    groups = _group_into_non_conflicting_bp(bp_list)
    return groups

def get_pk_bp_locs(groups):
    pk_bp_list = []
    pk_bp_locs = []
    for i, lists in enumerate(groups):
        if i == 0: 
            None
        else: 
            length = len(lists)
            for idx in range(length):
                bp = lists[idx]
                pk_bp_list.append(bp)
                pk_bp_locs.append(bp[0])
                pk_bp_locs.append(bp[1])             
    pk_bp_locs.sort()
    return pk_bp_locs, pk_bp_list

def get_pk_bp_struct(pk_bp_locs, dotbracket):
    pk_bp_struct = []
    for idx in pk_bp_locs:
        bracket = dotbracket[idx]
        pk_bp_struct.append(bracket)
    return pk_bp_struct

# rank PKs based on theoretically thermodynamic stability 

# ranking function operations: 
# add 1 for every additional base pair in a helix
# subtract 1 for every base pair adjacent to a separate helix 

def get_pk_rank(pk_bp_locs, dotbracket):
    pk_rank = 0
    for idx in pk_bp_locs:
        if (idx != 119) and (dotbracket[idx] == dotbracket[idx+1]):
            pk_rank += 0.5   
    for idx in pk_bp_locs: 
        if (idx != 119) and (dotbracket[idx] != dotbracket[idx+1]):
            if dotbracket[idx+1] == '.':
                pk_rank += 0
            else:
                pk_rank -= 1
    for idx in pk_bp_locs: 
        if (idx != 0) and (dotbracket[idx] != dotbracket[idx-1]):
            if dotbracket[idx-1] == '.':
                pk_rank += 0
            else:
                pk_rank -= 1
    return pk_rank

# rank PKs on consensus with other predictions

def get_bp_list(dotbracket):
    bp_list = convert_dotbracket_to_bp_list(dotbracket, allow_pseudoknots=True)
    return bp_list

def compare_bp_lists(bp_list1, bp_list2):
    bp_list_score = 0
    for bp1 in bp_list1: 
        for bp2 in bp_list2: 
            if bp1 == bp2: 
                bp_list_score += 1
        # divide by total number of base pairs in bp_list1 to normalize results
    return bp_list_score/len(bp_list1)

def get_consensus_scores(start_locs1, bp_lists1, start_locs2, bp_lists2):
    scores = []
    for i, loc1 in enumerate(start_locs1): 
        for idx, loc2 in enumerate(start_locs2):
            if loc1 == loc2:
                bp_list1 = bp_lists1[i]
                bp_list2 = bp_lists2[idx]
                bp_list_score = compare_bp_lists(bp_list1, bp_list2)
                scores.append(bp_list_score)
        else:
            scores.append(0)
    return scores

# create new dataframe with rankings

def get_df(starts, ends, sequences, dotbrackets, shape_scores, pk_bp_shape_scores, ranks, consensus_scores, pk_bp_consensus_scores):
    PK_list = zip(starts, ends, sequences, dotbrackets, shape_scores, pk_bp_shape_scores, ranks, consensus_scores, pk_bp_consensus_scores)
    df = pd.DataFrame(PK_list, columns = ['start', 'end', 'sequence', 'structure', 'shape_score', 'pk_bp_shape_score', 'rank', 'consensus_score', 'pk_bp_consensus_score'])
    ranked_df = df.sort_values('pk_bp_shape_score', ascending=False)
    return ranked_df

# get consensus score for only pk bps

def compare_bp_pks(struct1, struct2):
    bp_count = 0
    score = 0
    for i, char in enumerate(struct1):
        if char == '[' or char == ']':
            bp_count += 1
            if char == struct2[i]:
                score += 1
    return score/bp_count

def get_pk_bp_consensus_scores(starts, dotbrackets, starts2, dotbrackets2):
    bp_pk_consensus_scores = []
    for i, start in enumerate(starts):
        for i2, start2 in enumerate(starts2):
            if start == start2: 
                consensus_score = compare_bp_pks(dotbrackets[i], dotbrackets2[i2])
                bp_pk_consensus_scores.append(consensus_score)
        else: 
            bp_pk_consensus_scores.append(0)
    return bp_pk_consensus_scores
    
# put it all together

def score_pk_overall(csv, shape_file, csv2):
    df = get_csv(csv)
    starts, ends, sequences, dotbrackets = get_info(df)
    
    # get rough score for consensus with shape data for entire window
    
    full_shape = get_shape_data(shape_file)
    shapes = []
    for i, start in enumerate(starts):
        end = ends[i]
        shape_window = full_shape[start:end]
        shapes.append(shape_window)
    
    shape_scores = []
    for i, struct in enumerate(dotbrackets):
        shape = shapes[i]
        score = evaluate_L1_shape_score(struct, shape)
        shape_scores.append(score)

    # get score for shape consensus with only pk bps

    pk_bp_lists = []
    pk_bp_locs = []
    pk_bp_structs = []
    for i, struct in enumerate(dotbrackets):
        groups = get_groups(struct)
        pk_bp_loc, pk_bp_list = get_pk_bp_locs(groups)
        pk_bp_struct = get_pk_bp_struct(pk_bp_loc, struct)
        
        pk_bp_locs.append(pk_bp_loc)
        pk_bp_structs.append(pk_bp_struct)
        pk_bp_lists.append(pk_bp_list)
        
    pk_bp_shapes = []
    for i, locs in enumerate(pk_bp_locs):
        pk_bp_shapes_window = []
        shape_window = shapes[i]
        for idx in locs:
            shape = shape_window[idx]
            pk_bp_shapes_window.append(shape)
        pk_bp_shapes.append(pk_bp_shapes_window)
        
    pk_bp_shape_scores = []
    for i, struct in enumerate(pk_bp_structs):
        pk_bp_shape_window = pk_bp_shapes[i]
        score = evaluate_L1_shape_score(struct, pk_bp_shape_window)
        pk_bp_shape_scores.append(score)
    
    # get rough ranking for likelihood of PK
        
    ranks = []
    for i, struct in enumerate(dotbrackets): 
        rank = get_pk_rank(pk_bp_locs[i], struct)
        ranks.append(rank)
        
    # get consensus score with other predictions
    
    df2 = get_csv(csv2)
    starts2, ends2, sequences2, dotbrackets2 = get_info(df2)
    
    bp_lists1 = []
    for dotbracket in dotbrackets: 
        bp_list = get_bp_list(dotbracket)
        bp_lists1.append(bp_list)
        
    bp_lists2 = []
    for dotbracket in dotbrackets2: 
        bp_list = get_bp_list(dotbracket)
        bp_lists2.append(bp_list)
        
    consensus_scores = get_consensus_scores(starts, bp_lists1, starts2, bp_lists2)
    
    # get consensus scores for pk bps only 
    
    groups2 = []
    for dotbracket in dotbrackets2:
        groups = get_groups(dotbracket)
        groups2.append(groups)
        
    pk_bp_lists2 = []
    for group in groups2:
        pk_bp_loc, pk_bp_list = get_pk_bp_locs(group)
        pk_bp_lists2.append(pk_bp_list)
    
    pk_bp_consensus_scores = get_consensus_scores(starts, pk_bp_lists, starts2, pk_bp_lists2)
    
    # put it all together into a dataframe
        
    df = get_df(starts, ends, sequences, dotbrackets, shape_scores, pk_bp_shape_scores, ranks, consensus_scores, pk_bp_consensus_scores)
    return df

In [15]:
pknots_vs_threshknot = score_pk_overall('/home/gnye8/Desktop/PK_research/pipeline_results/pknots/pknots_output.csv', '/home/gnye8/Desktop/PK_research/SSRP_work/shape_data/incarnato_invivo_reactivity-Copy1.csv', '/home/gnye8/Desktop/PK_research/pipeline_results/threshknot/threshknot.csv')

In [14]:
knotty_vs_threshknot.to_csv('/home/gnye8/Desktop/PK_research/pipeline_results/knotty/knotty_vs_threshknot_analysis_scores.csv')

In [17]:
pknots_vs_threshknot.to_csv('/home/gnye8/Desktop/PK_research/pipeline_results/pknots/pknots_vs_threshknot_analysis_scores.csv')